In [51]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [3]:
import os
import psycopg2
conn = psycopg2.connect(host="localhost",database="postgres", user="postgres", password="postgres")
cursor = conn.cursor()

cursor.execute(create_table_query)
conn.commit()
cursor.close()
conn.close()

In [52]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r',encoding='utf8'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    
    print(body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [53]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage,VideoSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'video':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    


    # 回傳
    return returnArray

In [54]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

#圖片消息準備
myImageSendMessage = ImageSendMessage(original_content_url='https://i.imgur.com/jpVy8bN.jpg',
                                      preview_image_url='https://i.imgur.com/jpVy8bN.jpg' )

#引入按鍵模板
buttons_template_message_array = []
replyJsonPath = "素材/category/reply.json"
buttons_template_message_array = detect_json_array_to_new_message_array(replyJsonPath)


# 關注事件處理
# @handler.add(FollowEvent)
# def process_follow_event(event):
    
#     # 讀取並轉換
#     result_message_array =[]
#     replyJsonPath = "素材/關注/reply.json"
#     result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

#     # 消息發送
#     line_bot_api.reply_message(
#         event.reply_token,
#         result_message_array
#     )


@handler.add(FollowEvent)
def process_follow_event(event):
    # 取出消息內User的資料
    
    user_profile = line_bot_api.get_profile(event.source.user_id)
    
    lineRichMenuId = 'richmenu-37c601b9c38340b884d6f617cab84236'
    linkResult=line_bot_api.link_rich_menu_to_user(secretFileContentJson["self_user_id"], lineRichMenuId)
    # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
    
        
    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        buttons_template_message_array
    )


In [55]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)



In [56]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
# quickReplyTextSendMessage = TextSendMessage(text='What is a correct syntax to output \"Hello World\" in Python?\n(A) p(\"Hello World\")\n(B) print(\"Hello World\") \n(C) echo(\"Hello World\");\n(D) echo \"Hello World\"', quick_reply=quickReplyList)

In [57]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):

    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]
    replyJsonPath = "素材/"+event.message.text+"/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )


In [58]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 
Q = 1
@handler.add(PostbackEvent)

def process_postback_event(event):
    

    global Q
    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        result_message_array =[]

        replyJsonPath = '素材/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r', encoding='utf8').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
#         replyJsonPath = '素材/'+query_string_dict.get('menu')[0]+"/reply.json"
#         result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
        event.reply_token,
            TextMessage(
                text='感謝購買～'
            )
        )
    elif (event.postback.data.find('Q1') == 0):
        Q = 1
    elif (event.postback.data.find('rightAns') == 0):
        result_message_array =[]
        Q+=1
        replyJsonPath = "素材/Q"+ str(Q) + "/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif (event.postback.data.find('charged') == 0):
        line_bot_api.reply_message(
        event.reply_token,
            TextMessage(
                text='感謝購買～'
            )
        )
        
    else:
        line_bot_api.reply_message(
        event.reply_token,
            TextMessage(
                text='Wrong answer...'
            )
        )

In [59]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2020 18:28:35] "POST / HTTP/1.1" 200 -


{"events":[{"type":"unfollow","source":{"userId":"U1b31e6702cab04f084d9b597b2924fa8","type":"user"},"timestamp":1586082513966,"mode":"active"}],"destination":"U634eb7bc009dfbfcd76339ddf7484d23"}
{"events":[{"type":"follow","replyToken":"33d863190aea426894b9b512424ba93b","source":{"userId":"U1b31e6702cab04f084d9b597b2924fa8","type":"user"},"timestamp":1586082514833,"mode":"active"}],"destination":"U634eb7bc009dfbfcd76339ddf7484d23"}


127.0.0.1 - - [05/Apr/2020 18:28:36] "POST / HTTP/1.1" 200 -


{"events":[{"type":"message","replyToken":"7358b54e622c4422aad96434b9db0468","source":{"userId":"U1b31e6702cab04f084d9b597b2924fa8","type":"user"},"timestamp":1586082522921,"mode":"active","message":{"type":"text","id":"11729694709545","text":"Come on!"}}],"destination":"U634eb7bc009dfbfcd76339ddf7484d23"}


...

127.0.0.1 - - [05/Apr/2020 18:29:08] "POST / HTTP/1.1" 200 -
[2020-04-05 18:29:09,249] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Libr

{"events":[{"type":"message","replyToken":"74c9fb11167c45daaced0aa2e8cd6afd","source":{"userId":"U1b31e6702cab04f084d9b597b2924fa8","type":"user"},"timestamp":1586082547641,"mode":"active","message":{"type":"text","id":"11729696589407","text":"我要買課！"}}],"destination":"U634eb7bc009dfbfcd76339ddf7484d23"}


127.0.0.1 - - [05/Apr/2020 18:29:22] "POST / HTTP/1.1" 200 -


{"events":[{"type":"unfollow","source":{"userId":"U1b31e6702cab04f084d9b597b2924fa8","type":"user"},"timestamp":1586082562282,"mode":"active"}],"destination":"U634eb7bc009dfbfcd76339ddf7484d23"}
{"events":[{"type":"follow","replyToken":"241d7da02cf24650bc511f738771a036","source":{"userId":"U1b31e6702cab04f084d9b597b2924fa8","type":"user"},"timestamp":1586082563255,"mode":"active"}],"destination":"U634eb7bc009dfbfcd76339ddf7484d23"}


127.0.0.1 - - [05/Apr/2020 18:29:24] "POST / HTTP/1.1" 200 -


In [60]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

'\n\nApplication 運行（heroku版）\n\n'